In [2]:
import numpy as np
import subprocess
import collections
import random
import time
import sys
import os
from tqdm import tqdm
import json
import csv

#from network import FCDenseNet as Model
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable

# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
sys.path.append('./..')

from utils.utils import *
from data_generator.data_generator import RNASSDataGenerator
from data_generator.data_generator import Dataset_Cut_concat_new_canonicle as Dataset_FCN
from model.models import FCDenseNet
from utils.postprocess import postprocess_orig, postprocess_proposed



In [7]:
device_num = 0
pickle_path = '/media/ksj/nar_web_rna/test/test_red.pickle'

# nc seed 2 epoch 124 s 0
model_path = '/media/ksj/nar_web_rna/REDfold/model/redfold_model.pt'

# 나중에 바꾸기
num_of_device = 2
use_device_num = device_num
batch_n = 1
set_gamma = 0
set_rho= set_gamma+0.1
set_L1= 1

RNA_SS_data = collections.namedtuple('RNA_SS_data','name length seq_hot data_pair data_seq1 data_seq2')

os.environ["CUDA_VISIBLE_DEVICES"] = generate_visible_device(num_of_device)

device = torch.device('cuda:{}'.format(use_device_num))

Use_gpu= torch.cuda.is_available()

print('test data loading...')

test_data= RNASSDataGenerator(pickle_path,720)
test_len= len(test_data)
test_set= Dataset_FCN(test_data)

dataloader_test= DataLoader(dataset=test_set, batch_size=batch_n, shuffle=1, num_workers=12)

#- Network
model= FCDenseNet(in_channels=146,out_channels=1,
                initial_num_features=16,
                dropout=0,

                down_dense_growth_rates=(4,8,16,32),
                down_dense_bottleneck_ratios=None,
                down_dense_num_layers=(4,4,4,4),
                down_transition_compression_factors=1.0,

                middle_dense_growth_rate=32,
                middle_dense_bottleneck=None,
                middle_dense_num_layers=8,

                up_dense_growth_rates=(64,32,16,8),
                up_dense_bottleneck_ratios=None,
                up_dense_num_layers=(4,4,4,4))

optimizer= torch.optim.Adam(model.parameters())

# Model on GPU
if Use_gpu:
    model= model.to(device)

mod_state= torch.load(model_path, map_location=device)
model.load_state_dict(mod_state['state_dict'])

test data loading...


<All keys matched successfully>

In [8]:
torch.save(model.state_dict(), '/media/ksj/nar_web_rna/REDfold/model/redfold_model_new.pt', _use_new_zipfile_serialization=False)